# Test LLM with Events

## Primo test

In [2]:
import os
import openai
import pandas as pd

from IPython.display import Markdown, display

from langchain_community.llms import Ollama
from langchain_openai import ChatOpenAI

from event_tracking.components.event_analyzer import *
from event_tracking.config import RAW_DATA_DIR

llm_local = False
question = "Quali sono i 3 principali attività per un data scientist?"

if llm_local:
    llm = Ollama(model="gemma2:2b")
    # Semplice test diretto
    response_output = llm.invoke(question)
else:
    from dotenv import load_dotenv, find_dotenv
    _ = load_dotenv(find_dotenv()) # read local .env file
    openai.api_key = os.environ['OPENAI_API_KEY']

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

    # Semplice test diretto
    response = llm.invoke(question)
    response_output = response.content

display(Markdown(f"""
## Risposta LLM

{response_output}
"""))


## Risposta LLM

I 3 principali attività di un Data Scientist possono essere riassunte in questi punti:

**1. Analisi dei dati:**

* **Raccolta e pulizia dei dati:**  Il Data Scientist inizia la sua analisi con l'acquisizione e la preparazione di dati provenienti da diverse fonti. Questo richiede competenze di programmazione, database e pulizia di dati per rimuovere errori, duplicazioni o valori anomali.
* **Exploratory data analysis (EDA):**  Attraverso l'esplorazione dei dati, il Data Scientist cerca di comprendere la struttura, i pattern e le tendenze presenti nei dati. Questo include la generazione di grafici, statistiche descriptive e l'utilizzo di tecniche come la regressione per prevedere valori.
* **Modellazione:** L'obiettivo è identificare modelli che descrivano le relazioni tra variabili e fornire previsioni sui risultati futuri. Utilizza algoritmi statistici e machine learning per creare modelli, classificazione, previsione o clustering.

**2. Interpretazione dei risultati:** 

* **Presentazione dei risultati:**  I Data Scientist devono saper comunicare i loro risultati in modo chiaro e comprensibile a un pubblico non specializzato. Questo richiede una buona capacità di scrittura e presentazione, con l'uso di grafici e diagrammi per illustrare le relazioni tra variabili.
* **Valutazione del modello:**  Il Data Scientist deve verificare la qualità dei suoi modelli, utilizzando metriche predefinite come accuracy, precision, recall e F1-score. 
* **Analisi critica:**  L'analisi critica si basa sulla capacità di individuare potenziali problemi nel dataset o nel modello stesso. La capacità di comprendere i limiti del modello e le sfide future è fondamentale per il successo di un Data Scientist.

**3. Implementazione:**

* **Sviluppo di soluzione:**  Il Data Scientist deve utilizzare la sua conoscenza dei dati, dei modelli e delle tecniche per sviluppare soluzioni applicabili a problemi reali. Questo include la creazione di strumenti, dashboard e report per fornire insights utili alle decisioni aziendali.
* **Mantenimento del modello:** La manutenzione del modello è fondamentale per garantire la sua efficacia nel tempo.  Questo comporta l'aggiornamento dei dati, la correzione di errori e la verifica della performance del modello. 

Ricorda che queste sono solo alcune delle attività principali, il Data Scientist può anche svolgere altri compiti come la ricerca su nuove tecnologie o la collaborazione con altri professionisti per sviluppare soluzioni innovative.



In [3]:
file_path = os.path.join(RAW_DATA_DIR, 'my_calendar_db_20250508.parquet')

df = pd.read_parquet(file_path)

In [4]:
df_work = (
        df
        .loc[(df["calendar_name"] == "Pozz Work"), :]
        # .head()
    )

In [5]:
category_analysis = analyze_event_categories(df_work)

In [6]:
ollama_analyzer = create_ollama_analyzer(llm_initialized=llm)

C:\Users\cg08900\Documents\Pandora\github\event_tracking\event_tracking\components\event_analyzer.py:50: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(llm=llm_initialized, prompt=prompt_template)


In [7]:
# Esecuzione dell'analisi con Ollama
ollama_analysis = ollama_analyzer.run({
    "categories": str(category_analysis['counts']),
    "total_time": df_work['duration_minutes'].sum(),
    "top_categories": str(dict(sorted(category_analysis['percentage'].items(),
                                      key=lambda x: x[1],
                                      reverse=True)[:3]))
})

C:\Users\cg08900\AppData\Local\Temp\ipykernel_20924\2749007739.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ollama_analysis = ollama_analyzer.run({


In [8]:
display(Markdown(f"""
## Insights LLM sull'Utilizzo del Tempo

{ollama_analysis}
"""))


## Insights LLM sull'Utilizzo del Tempo

## Analisi del Profilo Temporale degli Eventi

Il tuo profilo temporale evidenzia una distribuzione significativa dei tuoi impegni su diversi settori. 

**1. Le principali categorie:**

* **"avm-property-value"**: occupa il maggior share del tempo analizzato (37.71%), suggerendo un'enfasi particolare su questa categoria. Questo potrebbe indicare un'attività specifica, come la gestione di immobili o la valutazione delle proprietà. 
    * **Suggerimento:** Considera se la tua attenzione all’ambito "avm-property-value" è ottimale in base alle tue priorità e obiettivi a breve/lungo termine. Se c'è spazio per un maggiore flusso di lavoro su altri ambiti, potrebbe essere utile spostare tempo e risorse in questi settori per una migliore performance complessiva.
* **"other"**: rappresenta il 9.58% del tempo analizzato, suggerendo l'assenza di un'area specifica con un livello di focus elevato. 
    * **Suggerimento:**  Verificare se ci sono attività che richiedono un’attenzione più costante, come la gestione dei contatti o la creazione di nuove strategie, per ottimizzare il flusso di lavoro.
* **"avm-meetings"**: occupa il 8.97% del tempo analizzato, evidenziando la necessità di organizzazione e priorità per questo tipo di attività. 
    * **Suggerimento:**  Valuta l'organizzazione delle tue riunioni previste: ottimizza le interazioni tra i partecipanti, limita il numero di meeting superflui, e concentrati su quelli più importanti. 


**2. Pattern significativi:**

* La differenza di focus tra "avm-property-value" e "other", con un aumento del tempo dedicato a "avm-meetings", potrebbe indicare una particolare fase di sviluppo o un'attività di consolidamento. 
* L’abbondanza di eventi relativi a **finbox** (gara-mcc, deploy-affordability, privati) e **side-project-tools-n-pipeline** suggerisce un focus sulla gestione delle attività finanziarie ed il potenziale per sviluppare nuove tecnologie o soluzioni.

**3. Suggerimenti per un migliore utilizzo del tempo:**

* **Priorità**:  Stabilisci una priorità per ogni categoria di eventi. Concentrati su quelle con maggiori probabilità di successo e che riflettono i tuoi obiettivi a breve/lungo termine.
* **Organizzazione**:  Utilizza strumenti come calendari, liste di attività o software di gestione del tempo per organizzare e pianificare le tue attività in modo più efficace.
* **Delega**: Se possibile, delegare alcune attività ad altri membri del team o alla società.
* **Stima Tempo**: Assegna tempi realistici alle attività per evitare sovraccarico e stress.


**Conclusioni:**

Analizzando il tuo profilo temporale, è evidente un'ampia gamma di attività con un focus specifico su "avm-property-value". Prendendo in considerazione i pattern significativi e seguendo gli suggerimenti, potrai ottimizzare il tuo tempo e raggiungere un maggiore successo. 






## Secondo test